In [37]:
import datetime
import heapq
import itertools

In [38]:
TIME_FORMAT="%Y-%m-%d-%H%M"
ONE_DAY=86400
ONE_HOUR=3600
ONE_MINUTE=60

In [54]:
def format_time(time):
    if isinstance(time, datetime.datetime):
        return time.strftime(TIME_FORMAT)
    else:
        print("Please pass a datetime object! I can't work under these conditions!!")


def format_period(period):
    def format_plural(n, unit):
        if n == 1:
            return "{} {}".format(n, unit)
        else:
            return "{} {}s".format(n, unit)
    return_str = ""
    if period < 0:
        return "OVERDUE"
    elif period > ONE_DAY:
        days = int(period/ONE_DAY)
        hours = int((period%ONE_DAY)/ONE_HOUR)
        return_str += format_plural(days, "day")
        if hours > 0:
            return_str += ", {}".format(format_plural(hours, "hour"))
        return return_str
    else:
        hours = int(period/ONE_HOUR)
        minutes = int((period%ONE_HOUR)/ONE_MINUTE)
        return_str += format_plural(hours, "hour")
        if minutes > 0:
            return_str += ", {}".format(format_plural(minutes, "minute"))
        return return_str

In [55]:
class Task():
    def __init__(self, name="Untitled-Task-{}".format(datetime.datetime.today().strftime(TIME_FORMAT)), 
                period=86400, 
                last=datetime.datetime.now(),
                comment="",
                category=""):
        self.name = name
        self.period = period
        self.history=[]
        self.due =  last + datetime.timedelta(seconds=period)
        self.comment = comment
        self.category = category
        print(self.name, self.period, self.due, self.comment, self.category)
    
    def check(self, ):
        self.due = datetime.datetime.now() + datetime.timedelta(seconds=self.period)
        self.history.append(datetime.datetime.now().strftime(TIME_FORMAT))
    
    def get_period(self,):
        return self.period
    
    def print_period(self, ):
        return format_period(self.get_period())

    def set_period(self, new_period):
        self.period = new_period
    
    def get_due(self):
        return self.due
    
    def print_due(self):
        return format_time(self.get_due())

    def set_due(self, new_due):
        self.due = new_due      

    def get_name(self, ):
        return self.name

    def set_name(self, new_name):
        self.name = new_name      

    def get_comment(self, ):
        return self.comment

    def set_comment(self, new_comment):
        self.comment = new_comment     

    def get_category(self, ):
        return self.category

    def set_category(self, new_category):
        self.category = new_category    

In [56]:
class TaskList():
    def __init__(self, taskList=[]):
        self.tasks = taskList
        self.count = itertools.count() 
        self.dict = {task.name:task for task in taskList} # TODO: update self.count as they are added

    def get_task(self, name):
        return self.dict.get(name)
    
    def show(self, ):
        heap = []
        for task in self.tasks:
            #print(task.name, task.due, task.print_period())
            tilldue = task.due - datetime.datetime.now()
            sec = int(tilldue.total_seconds())
            #print(tilldue)
            heapq.heappush(heap, (sec, task.name))
        returnlist = []
        heap.sort()
        for x in heap:
            name = x[1]
            due = x[0]
            due = format_period(due)
            returnlist.append("{}\t\t{}".format(name, due))
        print("\n".join(returnlist))
    
    def add(self, name="Untitled-Task-{}".format(datetime.datetime.today().strftime(TIME_FORMAT)), period=86400, last=datetime.datetime.now()):
        newTask = Task(name, period, last)
        self.tasks.append(newTask)
        self.dict[name] = newTask
        #TODO: Add self.count update
    
    def check(self, name):
        task = self.dict.get(name)
        if task:
            task.check()
        else:
            print("task {} not found.".format(task))

In [57]:
dolores=Task("Dolores", ONE_DAY * 15, datetime.datetime(2020,9,25,15))
jaqen=Task("Jaqen", ONE_DAY * 15, datetime.datetime(2020,9,16,15))
elan=Task("Elan", ONE_DAY * 10, datetime.datetime(2020,9,16,15))
dishes=Task("Dishes", ONE_DAY - 100, datetime.datetime.now() - datetime.timedelta(days=2))
pilates=Task("Pilates", ONE_DAY * 4)
yoga=Task("Yoga", ONE_DAY * 4, datetime.datetime(2020,9,27,11))
running=Task("Pilates", ONE_DAY * 4, datetime.datetime(2020,9,24,8))
bodyweight=Task("Bodyweight", ONE_DAY * 4, datetime.datetime(2020,9,25,17))
zhou=Task("Lemon Tree", ONE_DAY * 10, datetime.datetime.now() - datetime.timedelta(days=5))
backup=Task("Backup", ONE_DAY * 90)
sheets=Task("Sheets", ONE_DAY * 30, datetime.datetime(2020,9,12,12))
adventure=Task("Adventure", ONE_DAY * 7)
nails=Task("Nails", ONE_DAY * 7)
laundry=Task("Laundry", ONE_DAY * 7, datetime.datetime(2020,9,26,10))


Dolores 1296000 2020-10-10 15:00:00  
Jaqen 1296000 2020-10-01 15:00:00  
Elan 864000 2020-09-26 15:00:00  
Dishes 86300 2020-09-26 11:11:03.757324  
Pilates 345600 2020-10-01 11:12:43.463110  
Yoga 345600 2020-10-01 11:00:00  
Pilates 345600 2020-09-28 08:00:00  
Bodyweight 345600 2020-09-29 17:00:00  
Lemon Tree 864000 2020-10-02 11:12:43.759325  
Backup 7776000 2020-12-26 11:12:43.463110  
Sheets 2592000 2020-10-12 12:00:00  
Adventure 604800 2020-10-04 11:12:43.463110  
Nails 604800 2020-10-04 11:12:43.463110  
Laundry 604800 2020-10-03 10:00:00  


In [58]:
mytasks = [dolores, jaqen, elan, dishes, pilates, yoga, running, bodyweight]
for task in mytasks:
    print(task.name, task.print_period(), task.due)

Dolores 15 days 2020-10-10 15:00:00
Jaqen 15 days 2020-10-01 15:00:00
Elan 10 days 2020-09-26 15:00:00
Dishes 23 hours, 58 minutes 2020-09-26 11:11:03.757324
Pilates 4 days 2020-10-01 11:12:43.463110
Yoga 4 days 2020-10-01 11:00:00
Pilates 4 days 2020-09-28 08:00:00
Bodyweight 4 days 2020-09-29 17:00:00


In [59]:
heap = []
for task in mytasks:
    #print(task.name, task.due, task.print_period())
    tilldue = task.due - datetime.datetime.now()
    sec = int(tilldue.total_seconds())
    print(sec, task.name)
    heapq.heappush(heap, (sec, task.name))

heap

1136835 Dolores
359235 Jaqen
-72764 Elan
-86500 Dishes
345599 Pilates
344835 Yoga
74835 Pilates
193635 Bodyweight


[(-86500, 'Dishes'),
 (-72764, 'Elan'),
 (74835, 'Pilates'),
 (193635, 'Bodyweight'),
 (345599, 'Pilates'),
 (359235, 'Jaqen'),
 (344835, 'Yoga'),
 (1136835, 'Dolores')]

In [60]:
heap.sort()

In [61]:
mylist = TaskList(mytasks)

In [62]:
while heap:
    print(heapq.heappop(heap))

(-86500, 'Dishes')
(-72764, 'Elan')
(74835, 'Pilates')
(193635, 'Bodyweight')
(344835, 'Yoga')
(345599, 'Pilates')
(359235, 'Jaqen')
(1136835, 'Dolores')


In [63]:
mylist.show()

Dishes		OVERDUE
Elan		OVERDUE
Pilates		20 hours, 47 minutes
Bodyweight		2 days, 5 hours
Yoga		3 days, 23 hours
Pilates		3 days, 23 hours
Jaqen		4 days, 3 hours
Dolores		13 days, 3 hours


In [64]:
mylist.add("Lemon Tree", ONE_DAY * 10, datetime.datetime.now() - datetime.timedelta(days=5))

Lemon Tree 864000 2020-10-02 11:12:57.112397  


In [65]:
mylist.check('Dishes')

In [66]:
mylist.show()

Elan		OVERDUE
Pilates		20 hours, 47 minutes
Dishes		23 hours, 58 minutes
Bodyweight		2 days, 5 hours
Yoga		3 days, 23 hours
Pilates		3 days, 23 hours
Jaqen		4 days, 3 hours
Lemon Tree		4 days, 23 hours
Dolores		13 days, 3 hours


In [69]:
elan.print_period()

'10 days'

In [70]:
elan.get_comment()

''